In [2]:
import pandas as pd

import math
from typing import Any, Dict, List, Optional
from datetime import datetime
import rasterio
import contextily
import geopandas
import haversine as hs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests
from tqdm.notebook import tqdm

from huhuha.settings import RAW_DATA_DIR, DATA_DIR
from huhuha.ops_tiles import get_otm_tile
from huhuha.utils import get_elevation, random_float

tqdm.pandas()
sns.set_theme(style='whitegrid')

In [36]:
avalnache_data_path =  RAW_DATA_DIR / 'data-avalanche.csv'
df = pd.read_csv(avalnache_data_path)

In [37]:
summits = [
    'Dent Parrachée',
    # 'Punta Bagna',
    'La Norma',
]

In [38]:
# ograniczam zbior tylko do danych z z wybranych masywow i lat dla ktorych mam dane pogodowe
df = df[df['sommet'].isin(summits)]


# usuwam wiersze z brakiem informacji o masywach
df = df.dropna(subset=['massif'])

# usuwam wiersze z brakiem informacji o lokalizacji geograficznej
df = df.dropna(subset=['latitude', 'longitude'])
df = df.drop(df[(df.latitude == 0.0) | (df.longitude == 0.0)].index)

# dropuje wiersze gdzie brak informacji o dacie lawiny
df = df.dropna(subset=['date'])
df['date'] = pd.to_datetime(df['date'], unit='ms')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# df = df[df['year'] > 2006]|

In [39]:
df.shape

(82, 25)

In [40]:
df.groupby(['month']).count()

,id,date,massif,sommet,itineraire,description,orientation,caracteristique,origine_principale,origine_secondaire,...,longitude,thumbnail,imagesNb,isPrivate,tags,updated,altitude_depart,altitude_arrivee,year,day
month,,,,,,,,,,,,,,,,,,,,,
1,20,20,20,20,17,16,20,20,18,4,...,20,16,20,20,2,20,20,20,20,20
2,17,17,17,17,16,13,17,17,12,2,...,17,11,17,17,2,17,17,17,17,17
3,11,11,11,11,11,10,11,11,11,2,...,11,10,11,11,3,11,11,11,11,11
4,11,11,11,11,8,10,11,11,10,0,...,11,10,11,11,0,11,11,11,11,11
5,8,8,8,8,8,8,8,8,7,3,...,8,8,8,8,0,8,8,8,8,8
6,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,0,1,1,1,1,1
11,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,0,1,1,1,1,1
12,13,13,13,13,10,12,13,13,13,6,...,13,12,13,13,1,13,13,13,13,13


In [42]:
# przygotowanie dni lawinowyc

df_a = df[df['sommet'] == 'Dent Parrachée'][['id','date','year', 'month', 'day']]
df_a.head()

,id,date,year,month,day
154,1623007853941,2021-06-02 22:00:00,2021,6,2
164,1622040965780,2021-05-25 22:00:00,2021,5,25
545,1612033188232,2021-01-29 23:00:00,2021,1,29
591,1611822210227,2021-01-27 23:00:00,2021,1,27
704,1610987993192,2021-01-17 23:00:00,2021,1,17
